In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0818 15:28:24.324437 12668 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 15:28:24.328397 12668 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 15:28:24.329396 12668 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   1    2    3 ... 1997 1998 1999] TEST: [   0    6    7   17   19   28   30   33   36   38   40   42   43   45
   48   51   53   71   76   88   90   96   99  103  114  119  120  134
  140  141  142  147  149  152  153  163  164  165  181  185  190  196
  198  199  203  205  214  216  219  227  230  231  234  237  239  254
  260  265  268  269  270  276  289  292  294  296  305  315  327  332
  339  346  353  355  381  383  390  393  412  419  421  422  423  426
  427  432  433  435  440  441  443  455  465  466  474  477  488  489
  502  515  517  519  521  524  527  536  537  542  545  552  556  557
  567  569  579  581  590  591  596  615  624  625  626  629  634  639
  644  647  658  665  670  671  691  698  703  708  711  722  727  730
  732  736  740  749  750  754  757  765  768  770  773  787  807  808
  819  822  831  840  842  844  848  852  857  865  866  872  874  877
  879  880  887  889  896  898  900  906  917  919  924  926  929  936
  940  948  951  953  957  9

W0818 15:28:25.818441 12668 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.6166 - acc: 0.1439
Epoch 2/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5678 - acc: 0.1596
Epoch 3/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5356 - acc: 0.1733
Epoch 4/100
1600/1600 [==============================] - 14s 8ms/sample - loss: 0.5220 - acc: 0.1790
Epoch 5/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.5134 - acc: 0.1829
Epoch 6/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.5078 - acc: 0.1854
Epoch 7/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.5053 - acc: 0.1865
Epoch 8/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.4982 - acc: 0.1893
Epoch 9/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.4942 - acc: 0.1906
Epoch 10/100
1600/1600 [==============================] - 14s 9ms/sample - loss: 0.4929 - 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2474 - acc: 0.2754
Epoch 82/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2454 - acc: 0.2761
Epoch 83/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2433 - acc: 0.2769
Epoch 84/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2430 - acc: 0.2769
Epoch 85/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2422 - acc: 0.2773
Epoch 86/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2412 - acc: 0.2776
Epoch 87/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2398 - acc: 0.2781
Epoch 88/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2387 - acc: 0.2784
Epoch 89/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2376 - acc: 0.2790
Epoch 90/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2362 - acc: 

...

TRAIN: [   0    1    2 ... 1997 1998 1999] TEST: [   8    9   10   14   18   23   24   26   31   44   49   52   54   55
   56   58   61   64   70   73   83   85   86   87   89   97   98  100
  105  110  115  116  117  121  122  123  128  131  138  139  145  146
  150  151  156  162  174  183  186  189  193  197  202  204  208  224
  233  236  240  242  243  247  249  261  263  266  273  280  281  286
  297  307  309  310  312  316  320  323  326  334  337  342  348  350
  358  359  401  403  413  414  415  417  418  425  446  447  450  454
  459  460  463  470  478  479  482  497  501  503  506  507  509  523
  526  533  544  546  547  561  568  580  592  598  602  604  609  612
  617  618  643  646  650  654  655  661  669  674  680  681  685  686
  696  699  704  709  713  725  731  743  744  746  753  755  759  760
  767  772  780  785  791  792  795  798  800  801  802  804  811  815
  821  832  841  846  847  849  853  858  863  868  871  876  881  883
  884  885  890  897  902  9

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2800 - acc: 0.2631
Epoch 62/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2773 - acc: 0.2640
Epoch 63/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2751 - acc: 0.2647
Epoch 64/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2740 - acc: 0.2653
Epoch 65/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2709 - acc: 0.2664
Epoch 66/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2688 - acc: 0.2670
Epoch 67/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2676 - acc: 0.2672
Epoch 68/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2665 - acc: 0.2677
Epoch 69/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2642 - acc: 0.2689
Epoch 70/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2633 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4952 - acc: 0.1902
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4943 - acc: 0.1902
Epoch 11/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4869 - acc: 0.1933
Epoch 12/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4830 - acc: 0.1951
Epoch 13/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4818 - acc: 0.1952
Epoch 14/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4782 - acc: 0.1968
Epoch 15/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4719 - acc: 0.1991
Epoch 16/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4669 - acc: 0.2012
Epoch 17/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4623 - acc: 0.2029
Epoch 18/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4585 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2405 - acc: 0.2770
Epoch 90/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2402 - acc: 0.2771
Epoch 91/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2381 - acc: 0.2777
Epoch 92/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2363 - acc: 0.2785
Epoch 93/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2356 - acc: 0.2787
Epoch 94/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2336 - acc: 0.2794
Epoch 95/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2328 - acc: 0.2800
Epoch 96/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2327 - acc: 0.2797
Epoch 97/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2323 - acc: 0.2798
Epoch 98/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2305 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3500 - acc: 0.2374
Epoch 38/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3471 - acc: 0.2385
Epoch 39/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3390 - acc: 0.2411
Epoch 40/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3366 - acc: 0.2417
Epoch 41/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3332 - acc: 0.2426
Epoch 42/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3285 - acc: 0.2443
Epoch 43/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3262 - acc: 0.2451
Epoch 44/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3210 - acc: 0.2465
Epoch 45/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3192 - acc: 0.2474
Epoch 46/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3161 - acc: 

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_20 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_21 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_22 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 700, 200)         

In [10]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.36649565007323864, 0.39171004413797483, 0.3855978907456443, 0.38642805676794, 0.3923423276001009]
0.3845147938649798
Precision total
[0.5437842848219864, 0.5531843644575419, 0.5572278961823396, 0.5496243295858757, 0.5678375875364139]
0.5543316925168315
Recalls total
[0.552299106040401, 0.5653241463321786, 0.5696580777897521, 0.5612523041926502, 0.57778431130796]
0.5652635891325885
F1 total
[0.5393091260663508, 0.551304812550308, 0.5549867792320409, 0.5486184144175278, 0.5641977475170515]
0.5516833759566558
